# Prediction Methods

This file contains the code that tunes the hyperparameters for the k-NN, r-NN, and Entropy based confidence prediction methods.
It also evaluates the performance of these models on different datasets.

## Setup

In [ ]:
from __future__ import division
from __future__ import print_function
from os import path
import os, glob, torch,torchaudio, re
from python_speech_features import delta
from python_speech_features import mfcc

import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wav
import sys
import speech_dtw.qbe as qbe

from transformers import WavLMModel
from sklearn.decomposition import PCA

import math
from collections import Counter
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

sys.path.append("..")
sys.path.append(path.join("..", "utils"))

SAMPLE_RATE = 16000 
WAVLM_LAYER_INDEX = 6

device = "cpu"
model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(device).eval()

def cmvn(X):
    # X: [T, D] NumPy
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True)
    return (X - mu) / (sd + 1e-8)

def getWavLMFeatures(file): #A function which extracts MFCCs features from a given audio file
    sig, rate = torchaudio.load(file) #Reads the audio file, extracting the sample rate and signal data (as an array)

    #Check if sampled as correct sampling rate, if not - resample
    if rate != SAMPLE_RATE:
        print("Resampling", file ,"at 16kHz.\n")
        sig = torchaudio.functional.resample(sig, rate, SAMPLE_RATE)

    #Extracts layer 6 features
    sig = sig.to(device)
    with torch.inference_mode():
        out = model(sig, output_hidden_states=True)
        features = out.hidden_states[WAVLM_LAYER_INDEX].squeeze(0)  # [T, D] torch

    #Convert to numpy
    features = features.numpy()

    #Apply CMVN
    features = cmvn(features) #Applies cepstral mean and variance normalization to features
    
    return features

def getMinimumCost(queryFile, templateFile, FeatureType):
    #Loading the features
    if FeatureType == "MFCCs":
        queryFeatures = getMFCCsFeatures(queryFile) #Extract features for query data
    elif FeatureType == "WavLM":
        queryFeatures = getWavLMFeatures(queryFile) #Extract features for query data
        
    templateFeatures = torch.load(templateFile) #Load the template's feature file
    templateFeatures = templateFeatures["features"].numpy() #Extract the features as numpy arrays

    #Make both feature sets 2D, float64, contiguous:
    queryFeatures = np.ascontiguousarray(queryFeatures, dtype=np.float64)
    templateFeatures = np.ascontiguousarray(templateFeatures, dtype=np.float64)

    distance = qbe.dtw_sweep_min(queryFeatures, templateFeatures) #Calculate the minimum sweeping DTW distance between the two feature sets

    return distance

## k-NN

In [ ]:
def accuracies_knn(testFolder, templateFolder, ks):
    correct = {k: 0 for k in ks}
    total = 0
    class_counts = Counter()
    correct_per_class = {k: Counter() for k in ks}

    # Load templates
    template_files = list(Path(templateFolder).rglob("*.pt"))
    template_labels = {}
    for temp in template_files:
        parts = temp.stem.split("_")
        prefix = parts[0]
        if prefix.isdigit():
            template_labels[str(temp)] = int(prefix)

    for testFile in Path(testFolder).rglob("*.wav"):
        parts = testFile.stem.split("_")
        prefix = parts[0]
        true_label = int(prefix) if prefix.isdigit() else "No Number"
        class_counts[true_label] += 1
        total += 1

        # Compute distances
        distances = []
        for temp_path in template_labels:
            dist = getMinimumCost(str(testFile), temp_path, "WavLM")
            distances.append((dist, template_labels[temp_path]))

        # Sort
        distances.sort(key=lambda x: x[0])

        # Evaluate for each k
        for k in ks:
            if k <= 0: continue
            knn = distances[:k]
            labels = [lab for d, lab in knn]
            counts = Counter(labels)
            predicted_label = counts.most_common(1)[0][0]
            if predicted_label == true_label:
                correct[k] += 1
                correct_per_class[k][true_label] += 1

    results = {}
    for k in ks:
        accuracy = (correct[k] / total) * 100 if total > 0 else 0.0
        if k == 1:
            print(f"Classification Results for k=1:")
            print(f"Accuracy: {accuracy:.2f}%")
        results[k] = {
            "accuracy": accuracy,
            "class_counts": dict(class_counts),
            "correct_per_class": dict(correct_per_class[k])
        }
    return results

Usage:

In [ ]:
# Validation Data
ks = list(range(1, 11))
knnAccuracies = accuracies_knn("ValidationData/OnlyNumbers",
                               "TrainingData/TrainingFeatures/WavLMBase+/English",
                               ks=ks)

## r-NN

In [ ]:
def compute_metrics_for_r(testFile, template_labels, r, true_label):
    # Compute distances to all templates
    distances = []
    for temp_path, label in template_labels.items():
        dist = getMinimumCost(str(testFile), temp_path, "WavLM")
        distances.append((dist, label))

    # Find all templates within radius r
    rnn = [(d, lab) for d, lab in distances if d <= r]
    
    predicted_label = "No Number" if not rnn else Counter(lab for d, lab in rnn).most_common(1)[0][0]
    
    return true_label, predicted_label

def evaluate_rnn_metrics(testFolder, templateFolder, r_values):
    # Load all template files and their labels once
    template_files = list(Path(templateFolder).rglob("*.pt"))
    template_labels = {}
    for temp in template_files:
        parts = temp.stem.split("_")
        prefix = parts[0]
        if prefix.isdigit():
            template_labels[str(temp)] = int(prefix)

    test_files = list(Path(testFolder).rglob("*.wav"))
    true_labels = []
    for testFile in test_files:
        parts = testFile.stem.split("_")
        prefix = parts[0]
        true_label = int(prefix) if prefix.isdigit() else "No Number"
        true_labels.append(true_label)

    results = {}
    with ThreadPoolExecutor() as executor:
        for r in r_values:
            # Parallelize predictions for all test files
            predictions = list(executor.map(
                lambda tf: compute_metrics_for_r(tf, template_labels, r, true_labels[test_files.index(tf)]),
                test_files
            ))

            # Calculate overall metrics
            correct_predictions = 0
            true_positives = 0
            false_positives = 0
            false_negatives = 0
            true_negatives = 0
            total = len(test_files)

            for true_label, predicted_label in predictions:
                # Accuracy: exact match (same number or "No Number")
                if true_label == predicted_label:
                    correct_predictions += 1
                    if true_label != "No Number":
                        true_positives += 1
                    else:
                        true_negatives += 1
                else:
                    if predicted_label != "No Number":
                        false_positives += 1
                    if true_label != "No Number":
                        false_negatives += 1

            accuracy = correct_predictions / total * 100 if total > 0 else 0.0
            precision = true_positives / (true_positives + false_positives) * 100 if (true_positives + false_positives) > 0 else 0.0
            recall = true_positives / (true_positives + false_negatives) * 100 if (true_positives + false_negatives) > 0 else 0.0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

            # Print results for this r
            print(f"\nResults for r={r}:")
            print(f"Accuracy: {accuracy:.2f}%")
            print(f"Precision: {precision:.2f}%")
            print(f"Recall: {recall:.2f}%")
            print(f"F1 Score: {f1:.2f}%")

            results[r] = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1": f1
            }

    return results

Usage:

In [ ]:
# Validation Data
evaluate_rnn_metrics("ValidationData/WithNoNumClass", "TrainingData/TrainingFeatures/WavLMBase+/English/", np.arange(0.3, 0.45, 0.01) )

## Entropy

In [ ]:
def compute_metrics_for_entropy(testFile, template_labels, threshold, true_label):
    # Compute min distances to each class (0-9)
    min_dists = {lab: float('inf') for lab in range(10)}
    for temp_path, label in template_labels.items():
        dist = getMinimumCost(str(testFile), temp_path, "WavLM")
        if dist < min_dists[label]:
            min_dists[label] = dist

    dists = [min_dists[i] for i in range(10)]
    min_d = min(d for d in dists if d != float('inf'))
    sims = [math.exp(-(d - min_d)) if d != float('inf') else 0 for d in dists]
    sum_sim = sum(sims)
    probs = [s / sum_sim if sum_sim > 0 else 0 for s in sims]

    # Entropy
    entropy = -sum(p * math.log(p) for p in probs if p > 0)

    if entropy > threshold:
        predicted_label = "No Number"
    else:
        best_class = dists.index(min(dists))
        predicted_label = best_class
    
    return true_label, predicted_label

def evaluate_entropy_metrics(testFolder, templateFolder, threshold_values):
    # Load all template files and their labels once
    template_files = list(Path(templateFolder).rglob("*.pt"))
    template_labels = {}
    for temp in template_files:
        parts = temp.stem.split("_")
        prefix = parts[0]
        if prefix.isdigit():
            template_labels[str(temp)] = int(prefix)

    test_files = list(Path(testFolder).rglob("*.wav"))
    true_labels = []
    for testFile in test_files:
        parts = testFile.stem.split("_")
        prefix = parts[0]
        true_label = int(prefix) if prefix.isdigit() else "No Number"
        true_labels.append(true_label)

    results = {}
    with ThreadPoolExecutor() as executor:
        for threshold in threshold_values:
            # Parallelize predictions for all test files
            predictions = list(executor.map(
                lambda tf: compute_metrics_for_entropy(tf, template_labels, threshold, true_labels[test_files.index(tf)]),
                test_files
            ))

            # Calculate overall metrics
            true_positives = 0
            false_positives = 0
            false_negatives = 0
            true_negatives = 0
            total = len(test_files)

            for true_label, predicted_label in predictions:
                is_true_number = true_label != "No Number"
                is_pred_number = predicted_label != "No Number"
                
                if is_true_number and is_pred_number:
                    true_positives += 1
                elif is_true_number and not is_pred_number:
                    false_negatives += 1
                elif not is_true_number and is_pred_number:
                    false_positives += 1
                else:
                    true_negatives += 1

            accuracy = (true_positives + true_negatives) / total * 100 if total > 0 else 0.0
            precision = true_positives / (true_positives + false_positives) * 100 if (true_positives + false_positives) > 0 else 0.0
            recall = true_positives / (true_positives + false_negatives) * 100 if (true_positives + false_negatives) > 0 else 0.0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

            # Print results for this threshold
            print(f"\nResults for threshold={threshold}:")
            print(f"Accuracy: {accuracy:.2f}%")
            print(f"Precision: {precision:.2f}%")
            print(f"Recall: {recall:.2f}%")
            print(f"F1 Score: {f1:.2f}%")

            results[threshold] = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1": f1
            }

    return results

Usage:

In [ ]:
evaluate_entropy_metrics("ValidationData/WithNoNumClass", "TrainingData/TrainingFeatures/WavLMBase+/Child/Jibo", np.arange(2.1, 2.6, 0.1))

# Prediction Method Performance

Minimum Cost:

In [ ]:
# Validation Data
minCostValidationAccuracy = accuracies_knn("ValidationData/OnlyNumbers",
                               "TrainingData/TrainingFeatures/WavLMBase+/English",
                               [1])

In [ ]:
# Testing Data
minCostTestingAccuracy = accuracies_knn("TestingData/OnlyNumbers",
                               "TrainingData/TrainingFeatures/WavLMBase+/English",
                               [1])

k-NN with k=3:

In [ ]:
# Validation Data
knnValidationAccuracy = accuracies_knn("ValidationData/OnlyNumbers",
                               "TrainingData/TrainingFeatures/WavLMBase+/English",
                               [3])

In [ ]:
# Testing Data
knnTestingAccuracy = accuracies_knn("TestingData/OnlyNumbers",
                               "TrainingData/TrainingFeatures/WavLMBase+/English",
                               [3])

r-NN with r=0.34

In [ ]:
# Validation Data
rnnValidationAccuracy = evaluate_rnn_metrics("ValidationData/WithNoNumClass", "TrainingData/TrainingFeatures/WavLMBase+/English/", np.arange(0.36, 0.37, 0.01) )

In [ ]:
# Testing Data
rnnTestingAccuracy = evaluate_rnn_metrics("TestingData/WithNoNumClass", "TrainingData/TrainingFeatures/WavLMBase+/English/", np.arange(0.36, 0.37, 0.01) )

# Final Prediction Function's Accuracy

English:

In [ ]:
accuracyEnglish =  accuracies_knn("TestingData/OnlyNumbers/English", "TrainingData/TrainingFeatures/WavLMBase+/Child/Jibo", [3])

Afrikaans:

In [ ]:
accuracyAfrikaans =  accuracies_knn("TestingData/OnlyNumbers/Afrikaans", "TrainingFeatures/WavLMBase+/Afrikaans", [3])